# Piškorky algoritmus minmax

In [39]:
import numpy as np

In [40]:
import numpy as np

class State:
    """ Zachycení stavu partie
    gameplan - dvourozměrné pole 3x3
             - 0 - prázdné poole
             - 1 - X
             - 2 - O
    player - hráč, který je v partii na tahu

    current_player - hráč, který je na tahu v daném stavu při prohledávání stavového prostoru
    depth          - hloubka prohledávání stavového prostoru
    max_depth      - maximální délka prohledávání
    """

    generated = 0

    def __init__(self, gameplan, player, current_player=None, depth=0, max_depth=100):
        self.gameplan = gameplan
        self.player = player
        if current_player is None:
            self.current_player = player
        else:
            self.current_player = current_player
        self.depth = depth
        self.max_depth = max_depth
        
        State.generated += 1

    def terminal_test(self):
        for i in range(3):
            if np.array_equal(self.gameplan[i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[i], [2, 2, 2]):
                return 2
            if np.array_equal(self.gameplan[:, i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[:, i], [2, 2, 2]):
                return 2

        if np.array_equal(self.gameplan.diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(self.gameplan.diagonal(), [2, 2, 2]):
            return 2
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [2, 2, 2]):
            return 2

        return 0

    def utility(self, result):
        if result == 0:
            return 0
        elif result == self.player:
            return 1
        else:
            return -1

    def possible_actions(self):
        possible_actions = []
        for i in range(3):
            for j in range(3):
                if self.gameplan[i][j] == 0:
                    possible_actions.append((i, j))
        return possible_actions

    def expand(self, select_action):
        if select_action[0] not in range(3) or select_action[1] not in range(3):
            return None
        if self.gameplan[select_action[0], select_action[1]] != 0:
            return None
        
        new_array = np.copy(self.gameplan)
        new_array[select_action[0], select_action[1]] = self.current_player
        return State(new_array, 
                     self.player, 
                     self.next_current_player(), 
                     self.depth + 1, 
                     max_depth=self.max_depth)

    def minmax(self, strategy="max", alfa=float('-inf'), beta=float('inf')):
        result = self.terminal_test()
        if result != 0:
            return self.utility(result), None

        if self.depth >= self.max_depth:
            return 0, None

        if strategy == "max":
            selected_utilization_value = float('-inf')
            next_strategy = "min"
        else:
            selected_utilization_value = float('inf')
            next_strategy = "max"

        actions = self.possible_actions()
        if not actions:
            return 0, None

        selected_action = actions[0]

        for action in actions:
            expanded_state = self.expand(action)
            result = expanded_state.terminal_test()

            if result != 0:
                return expanded_state.utility(result), action

            if len(expanded_state.possible_actions()) == 0:
                utilization = 0
            else:
                utilization, _ = expanded_state.minmax(next_strategy, alfa, beta)

            if strategy == "max":
                if utilization > selected_utilization_value:
                    selected_utilization_value = utilization
                    selected_action = action

                if selected_utilization_value >= beta:
                    return selected_utilization_value, selected_action
                if selected_utilization_value > alfa:
                    alfa = selected_utilization_value
            else:
                if utilization < selected_utilization_value:
                    selected_utilization_value = utilization
                    selected_action = action

                if selected_utilization_value <= alfa:
                    return selected_utilization_value, selected_action
                if selected_utilization_value < beta:
                    beta = selected_utilization_value

        return selected_utilization_value, selected_action

    def next_current_player(self):
        return 3 - self.current_player

    def next_player(self):
        return 3 - self.player


In [41]:
# vytvoreni pocatecniho stavu partie
# herni plan je prazdny
# na tahu je hrac 1
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1,
              max_depth=5)  # přidané omezení hloubky

In [42]:
# vypsani pocatecniho stavu
print(state.gameplan)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [43]:
# Sledujte čas a počty generovaných stavů s postupující partií

# Cyklus pro hru, kde hrají proti sobě dvě kopie algoritmu
while True:
    # Kontrola, zda není partie u konce
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Kontrola, zda není remíza
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # tah hráče
    print(f"=====================\nPlayer {state.player}")
    utility, player_action = state.minmax("max")
    
    # Проверка, если ход не найден (None), вывести сообщение и завершить
    if player_action is None:
        print("No valid move available.")
        break

    print(f"Select action: {player_action}")
    state = state.expand(player_action)
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # přepnutí partie na druhého hráče
    state.player = state.next_player()

Player 1
Select action: (0, 0)
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Generated states 1015.
Player 2
Select action: (0, 1)
[[1 2 0]
 [0 0 0]
 [0 0 0]]
Generated states 209.
Player 1
Select action: (0, 2)
[[1 2 1]
 [0 0 0]
 [0 0 0]]
Generated states 110.
Player 2
Select action: (1, 0)
[[1 2 1]
 [2 0 0]
 [0 0 0]]
Generated states 17.
Player 1
Select action: (1, 1)
[[1 2 1]
 [2 1 0]
 [0 0 0]]
Generated states 6.
Player 2
No valid move available.


# Úkol
Do algoritmu přidejte omezeni na maximalní prohledávanou hloubku.